In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
# MPC import
import numpy as np
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
# MPC parameters
Ts = 0.05       # Sampling time (20 Hz)
sim_time = 7.0  # Simulation time (should stabilize in < 7s)
H = 3.0         # MPC horizon (3 seconds)

# Create rocket and MPC controller
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl.new_controller(rocket, Ts, H)

print(f"MPC Controllers initialized:")
print(f"  Sampling time: {Ts}s")
print(f"  Horizon: {H}s ({int(H/Ts)} steps)")
print(f"  Simulation time: {sim_time}s")

## Test 1: Stabilize from 5 m/s in X direction

**Goal**: Start with 5 m/s velocity in x direction, stabilize to zero velocity.

**Expected outcome**:
- Settling time < 7 seconds
- Final velocity vx ≈ 0 m/s
- Overshoot < 20%
- Roll and pitch angles stay small (|alpha|, |beta| < 10°)

In [ ]:
# Initial state: 5 m/s in x direction
# State vector: x = [wx, wy, wz, alpha, beta, gamma, vx, vy, vz, x, y, z]
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0])

# Run MPC simulation
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)

# Print results
print(f"\nTest 1 Results (5 m/s in x):")
print(f"  Initial state: vx={x0[6]:.2f} m/s")
print(f"  Final state:")
print(f"    vx={x_cl[6, -1]:.3f} m/s (should be ≈0)")
print(f"    vy={x_cl[7, -1]:.3f} m/s (should be ≈0)")
print(f"    vz={x_cl[8, -1]:.3f} m/s (should be ≈0)")
print(f"    beta={np.rad2deg(x_cl[4, -1]):.2f}° (should be < 10°)")
print(f"    alpha={np.rad2deg(x_cl[3, -1]):.2f}° (should be < 10°)")
print(f"  Position: x={x_cl[9, -1]:.2f}m, y={x_cl[10, -1]:.2f}m, z={x_cl[11, -1]:.2f}m")

# Visualize
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol)

## Test 2: Stabilize from 5 m/s in Y direction

**Goal**: Start with 5 m/s velocity in y direction, stabilize to zero velocity.

**Expected outcome**: Same as Test 1, but for y direction.

In [ ]:
# Initial state: 5 m/s in y direction
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0])

# Run MPC simulation
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)

# Print results
print(f"\nTest 2 Results (5 m/s in y):")
print(f"  Initial state: vy={x0[7]:.2f} m/s")
print(f"  Final state:")
print(f"    vx={x_cl[6, -1]:.3f} m/s (should be ≈0)")
print(f"    vy={x_cl[7, -1]:.3f} m/s (should be ≈0)")
print(f"    vz={x_cl[8, -1]:.3f} m/s (should be ≈0)")
print(f"    beta={np.rad2deg(x_cl[4, -1]):.2f}° (should be < 10°)")
print(f"    alpha={np.rad2deg(x_cl[3, -1]):.2f}° (should be < 10°)")
print(f"  Position: x={x_cl[9, -1]:.2f}m, y={x_cl[10, -1]:.2f}m, z={x_cl[11, -1]:.2f}m")

# Visualize
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol)

## Test 3: Stabilize from 5 m/s upward (Z direction)

**Goal**: Start with 5 m/s upward velocity, stabilize to zero velocity.

**Expected outcome**: 
- Settling time < 7 seconds
- Final velocity vz ≈ 0 m/s
- Throttle stays within bounds [40%, 80%]

In [ ]:
# Initial state: 5 m/s upward (positive z)
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0])

# Run MPC simulation
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)

# Print results
print(f"\nTest 3 Results (5 m/s upward):")
print(f"  Initial state: vz={x0[8]:.2f} m/s")
print(f"  Final state:")
print(f"    vx={x_cl[6, -1]:.3f} m/s (should be ≈0)")
print(f"    vy={x_cl[7, -1]:.3f} m/s (should be ≈0)")
print(f"    vz={x_cl[8, -1]:.3f} m/s (should be ≈0)")
print(f"  Throttle range: Pavg ∈ [{u_cl[2, :].min():.1f}%, {u_cl[2, :].max():.1f}%] (should be [40, 80])")
print(f"  Position: x={x_cl[9, -1]:.2f}m, y={x_cl[10, -1]:.2f}m, z={x_cl[11, -1]:.2f}m")

# Visualize
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol)

## Test 4: Combined Initial Conditions

**Goal**: Start with velocities in all directions + roll angle, stabilize to zero.

**Expected outcome**: 
- All subsystems work together to stabilize
- Settling time < 7 seconds
- All final states ≈ 0

In [ ]:
# Initial state: Combined disturbances
# vx = 3 m/s, vy = 2 m/s, vz = 1 m/s, gamma = 15° roll
x0 = np.array([
    0.0,            # wx
    0.0,            # wy
    0.0,            # wz
    0.0,            # alpha
    0.0,            # beta
    np.deg2rad(15), # gamma (15° roll)
    3.0,            # vx
    2.0,            # vy
    1.0,            # vz
    0.0,            # x
    0.0,            # y
    0.0             # z
])

# Run MPC simulation
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)

# Print results
print(f"\nTest 4 Results (Combined):")
print(f"  Initial state: vx={x0[6]:.1f}, vy={x0[7]:.1f}, vz={x0[8]:.1f} m/s, gamma={np.rad2deg(x0[5]):.1f}°")
print(f"  Final state:")
print(f"    vx={x_cl[6, -1]:.3f} m/s (should be ≈0)")
print(f"    vy={x_cl[7, -1]:.3f} m/s (should be ≈0)")
print(f"    vz={x_cl[8, -1]:.3f} m/s (should be ≈0)")
print(f"    gamma={np.rad2deg(x_cl[5, -1]):.3f}° (should be ≈0)")
print(f"    beta={np.rad2deg(x_cl[4, -1]):.2f}° (should be < 10°)")
print(f"    alpha={np.rad2deg(x_cl[3, -1]):.2f}° (should be < 10°)")
print(f"  Position: x={x_cl[9, -1]:.2f}m, y={x_cl[10, -1]:.2f}m, z={x_cl[11, -1]:.2f}m")

# Visualize
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol)